# Tech Challenge - Fine-tuning para Produtos Amazon

**Objetivo**: Executar fine-tuning de um foundation model usando o dataset AmazonTitles-1.3MM para gerar descrições de produtos baseadas em títulos.

**Dataset**: Utilizaremos o arquivo `trn.json.gz` que contém títulos e descrições de produtos da Amazon.

**Modelo Escolhido**: Llama 3-8B com Unsloth para otimização de treinamento.

---

## 📋 Índice
1. [Configuração Inicial](#1-configuracao-inicial)
2. [Exploração dos Dados](#2-exploracao-dos-dados)
3. [Preparação do Dataset](#3-preparacao-do-dataset)
4. [Teste do Modelo Base](#4-teste-do-modelo-base)
5. [Fine-tuning](#5-fine-tuning)
6. [Teste do Modelo Treinado](#6-teste-do-modelo-treinado)
7. [Demonstração Interativa](#7-demonstracao-interativa)

---

## 1. Configuração Inicial

### 1.1 Montagem do Google Drive
Primeiro, vamos montar o Google Drive para acessar e salvar nossos arquivos.

In [ ]:
from google.colab import drive
import os

# Monta o Google Drive
drive.mount('/content/drive')

# Define o diretório de trabalho (usando o mesmo diretório onde está o arquivo de dados)
WORK_DIR = '/content/drive/MyDrive/FineTunning/TechChallenge03'
os.makedirs(WORK_DIR, exist_ok=True)

print(f"✅ Google Drive montado com sucesso!")
print(f"📁 Diretório de trabalho: {WORK_DIR}")

# Verifica se o diretório existe e lista os arquivos
if os.path.exists(WORK_DIR):
    files_in_dir = os.listdir(WORK_DIR)
    print(f"📋 Arquivos no diretório: {files_in_dir}")
else:
    print(f"⚠️ Diretório não existe, será criado: {WORK_DIR}")

### 1.2 Instalação das Dependências

Instalamos as bibliotecas necessárias:
- **Unsloth**: Otimização para fine-tuning eficiente
- **Transformers**: Biblioteca principal para modelos de linguagem
- **Datasets**: Para manipulação de datasets
- **TRL**: Para treinamento de modelos de linguagem

In [ ]:
# Instalação das dependências principais
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets torch

print("✅ Todas as dependências foram instaladas com sucesso!")

### 1.3 Importação das Bibliotecas e Configurações Iniciais

In [ ]:
# Imports necessários
import json
import gzip
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import torch
from transformers import TrainingArguments, TextStreamer
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")
print(f"🔥 CUDA disponível: {torch.cuda.is_available()}")
print(f"💾 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Não disponível'}")

### 1.4 Configurações do Modelo e Treinamento

In [ ]:
# Configurações principais
CONFIG = {
    # Configurações do modelo
    'model_name': "unsloth/llama-3-8b-bnb-4bit",
    'max_seq_length': 2048,
    'dtype': None,  # Será determinado automaticamente
    'load_in_4bit': True,
    
    # Configurações do dataset
    'data_file': '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz',
    'sample_size': 10000,  # Número de amostras para treinamento (pode ajustar)
    'test_size': 100,  # Número de amostras para teste
    
    # Configurações do fine-tuning
    'lora_r': 16,
    'lora_alpha': 16,
    'lora_dropout': 0,
    'max_steps': 100,  # Ajuste conforme necessário
    'learning_rate': 2e-4,
    'batch_size': 2,
    'gradient_accumulation_steps': 4,
    
    # Caminhos - usando o mesmo diretório base
    'base_dir': '/content/drive/MyDrive/FineTunning/TechChallenge03',
    'output_dir': '/content/drive/MyDrive/FineTunning/TechChallenge03/outputs',
    'model_save_path': '/content/drive/MyDrive/FineTunning/TechChallenge03/amazon_model',
}

# Criação dos diretórios
os.makedirs(CONFIG['base_dir'], exist_ok=True)
os.makedirs(CONFIG['output_dir'], exist_ok=True)
os.makedirs(CONFIG['model_save_path'], exist_ok=True)

print("⚙️ Configurações definidas:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 2. Exploração dos Dados

### 2.1 Upload do Arquivo de Dados

Primeiro, você precisa fazer upload do arquivo `trn.json.gz` para o Colab.
Execute a célula abaixo e faça upload do arquivo quando solicitado.

In [ ]:
import os

# Define o caminho para o arquivo no Google Drive
DATA_FILE_PATH = '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz'

# Verifica se o arquivo existe
if os.path.exists(DATA_FILE_PATH):
    print("✅ Arquivo trn.json.gz encontrado no Google Drive!")
    print(f"📁 Caminho: {DATA_FILE_PATH}")
    
    # Verifica o tamanho do arquivo
    file_size = os.path.getsize(DATA_FILE_PATH)
    print(f"📊 Tamanho do arquivo: {file_size / (1024*1024):.1f} MB")
else:
    print("❌ Arquivo não encontrado no caminho especificado.")
    print(f"❌ Caminho verificado: {DATA_FILE_PATH}")
    print("💡 Certifique-se de que o arquivo trn.json.gz está no diretório correto do Google Drive.")
    DATA_FILE_PATH = None

### 2.2 Carregamento e Análise Inicial dos Dados

Vamos carregar o dataset e analisar sua estrutura para entender melhor os dados com que estamos trabalhando.

In [ ]:
def load_amazon_data(file_path, sample_size=None):
    """
    Carrega os dados do arquivo JSON comprimido
    
    Args:
        file_path: Caminho para o arquivo trn.json.gz
        sample_size: Número de amostras a carregar (None para carregar tudo)
    
    Returns:
        Lista de dicionários com os dados
    """
    data = []
    
    print(f"📖 Carregando dados de {file_path}...")
    
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if sample_size and i >= sample_size:
                    break
                    
                try:
                    json_obj = json.loads(line.strip())
                    data.append(json_obj)
                except json.JSONDecodeError:
                    continue
                    
                # Progress update
                if (i + 1) % 1000 == 0:
                    print(f"  Carregadas {i + 1} amostras...")
    
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        return []
    
    print(f"✅ Dados carregados com sucesso! Total: {len(data)} amostras")
    return data

# Carrega uma amostra dos dados para análise inicial
if DATA_FILE_PATH:
    sample_data = load_amazon_data(DATA_FILE_PATH, sample_size=5000)
else:
    print("❌ Arquivo de dados não disponível. Execute a célula de upload primeiro.")

In [ ]:
# Análise da estrutura dos dados
if sample_data:
    print("🔍 ANÁLISE DA ESTRUTURA DOS DADOS")
    print("=" * 50)
    
    # Exemplo de uma amostra
    print("📝 Exemplo de uma amostra:")
    sample_item = sample_data[0]
    for key, value in sample_item.items():
        print(f"  {key}: {value}")
    
    print("\n" + "=" * 50)
    
    # Estatísticas gerais
    print(f"📊 ESTATÍSTICAS GERAIS:")
    print(f"  Total de amostras carregadas: {len(sample_data)}")
    
    # Verifica campos obrigatórios
    required_fields = ['title', 'content']
    valid_samples = []
    
    for item in sample_data:
        if all(field in item and item[field] for field in required_fields):
            valid_samples.append(item)
    
    print(f"  Amostras válidas (com título e conteúdo): {len(valid_samples)}")
    print(f"  Taxa de amostras válidas: {len(valid_samples)/len(sample_data)*100:.1f}%")
    
    # Armazena as amostras válidas
    sample_data = valid_samples
    
else:
    print("❌ Nenhum dado foi carregado.")

In [ ]:
# Análise detalhada dos textos
if sample_data:
    print("📏 ANÁLISE DE COMPRIMENTO DOS TEXTOS")
    print("=" * 50)
    
    # Calcula estatísticas de comprimento
    title_lengths = [len(item['title']) for item in sample_data]
    content_lengths = [len(item['content']) for item in sample_data]
    
    title_words = [len(item['title'].split()) for item in sample_data]
    content_words = [len(item['content'].split()) for item in sample_data]
    
    print("📝 Comprimento em caracteres:")
    print(f"  Títulos - Mín: {min(title_lengths)}, Máx: {max(title_lengths)}, Média: {np.mean(title_lengths):.1f}")
    print(f"  Conteúdo - Mín: {min(content_lengths)}, Máx: {max(content_lengths)}, Média: {np.mean(content_lengths):.1f}")
    
    print("\n🔤 Comprimento em palavras:")
    print(f"  Títulos - Mín: {min(title_words)}, Máx: {max(title_words)}, Média: {np.mean(title_words):.1f}")
    print(f"  Conteúdo - Mín: {min(content_words)}, Máx: {max(content_words)}, Média: {np.mean(content_words):.1f}")
    
    # Exemplos de diferentes tamanhos
    print("\n📋 EXEMPLOS DE PRODUTOS:")
    print("=" * 50)
    
    # Título mais curto
    shortest_idx = title_lengths.index(min(title_lengths))
    print(f"🔸 Título mais curto ({len(sample_data[shortest_idx]['title'])} chars):")
    print(f"  Título: {sample_data[shortest_idx]['title']}")
    print(f"  Conteúdo: {sample_data[shortest_idx]['content']}")
    
    print("\n" + "-" * 30)
    
    # Título mais longo
    longest_idx = title_lengths.index(max(title_lengths))
    print(f"🔸 Título mais longo ({len(sample_data[longest_idx]['title'])} chars):")
    print(f"  Título: {sample_data[longest_idx]['title']}")
    print(f"  Conteúdo: {sample_data[longest_idx]['content'][:200]}...")
    
    print("\n" + "-" * 30)
    
    # Exemplo aleatório
    import random
    random_idx = random.randint(0, len(sample_data)-1)
    print(f"🔸 Exemplo aleatório:")
    print(f"  Título: {sample_data[random_idx]['title']}")
    print(f"  Conteúdo: {sample_data[random_idx]['content']}")
    
else:
    print("❌ Nenhum dado disponível para análise.")

In [ ]:
# Visualizações
if sample_data:
    print("📊 CRIANDO VISUALIZAÇÕES DOS DADOS")
    print("=" * 50)
    
    # Configuração do matplotlib
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Análise do Dataset Amazon Products', fontsize=16, fontweight='bold')
    
    # Gráfico 1: Distribuição do comprimento dos títulos
    axes[0,0].hist(title_lengths, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0,0].set_title('Distribuição - Comprimento dos Títulos (caracteres)')
    axes[0,0].set_xlabel('Número de caracteres')
    axes[0,0].set_ylabel('Frequência')
    axes[0,0].grid(True, alpha=0.3)
    
    # Gráfico 2: Distribuição do comprimento do conteúdo
    axes[0,1].hist(content_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,1].set_title('Distribuição - Comprimento do Conteúdo (caracteres)')
    axes[0,1].set_xlabel('Número de caracteres')
    axes[0,1].set_ylabel('Frequência')
    axes[0,1].grid(True, alpha=0.3)
    
    # Gráfico 3: Distribuição de palavras nos títulos
    axes[1,0].hist(title_words, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1,0].set_title('Distribuição - Palavras nos Títulos')
    axes[1,0].set_xlabel('Número de palavras')
    axes[1,0].set_ylabel('Frequência')
    axes[1,0].grid(True, alpha=0.3)
    
    # Gráfico 4: Relação entre título e conteúdo
    axes[1,1].scatter(title_lengths, content_lengths, alpha=0.5, color='purple', s=10)
    axes[1,1].set_title('Relação: Título vs Conteúdo (caracteres)')
    axes[1,1].set_xlabel('Comprimento do título')
    axes[1,1].set_ylabel('Comprimento do conteúdo')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizações criadas com sucesso!")
    
else:
    print("❌ Nenhum dado disponível para visualização.")